In [36]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle
import seaborn as sns
import matplotlib

from datetime import timedelta

import matplotlib.pyplot as plt

# Normalization/Standardization
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, Conv1D, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.saving import save_model

# 경고 무시 코드 추가
import warnings
warnings.filterwarnings('ignore')

def pltconfig_default() :
  sns.reset_defaults()
  %matplotlib inline

pltconfig_default()

matplotlib.rcParams

matplotlib.rcParams['font.family']

current_font_list = matplotlib.rcParams['font.family']

font_path = 'C:\\Windows\\Fonts\\batang.ttc'

kfont = matplotlib.font_manager.FontProperties(fname=font_path).get_name()

matplotlib.rcParams['font.family'] = [kfont] + current_font_list

In [37]:
St_NotEncode_data = pd.read_pickle("StandardScalar_final_data")

print(St_NotEncode_data.shape)

(17422, 9)


In [38]:
# Feature와 Label 분리하기 (풍속, 습도, 일조시간, 전운량)
def Feature_Label(datafile) :
    X = datafile.iloc[:,[1,3,4,5]]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SNE_X, SNE_y = Feature_Label(St_NotEncode_data)
print(SNE_X.shape, SNE_y.shape)
SNE_X_train, SNE_X_test, SNE_y_train, SNE_y_test = train_test_split(SNE_X, SNE_y, test_size=0.2, random_state=10, shuffle=False)
print(SNE_X_train.shape, SNE_X_test.shape, SNE_y_train.shape, SNE_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 4) (17422,)
(13937, 4) (3485, 4) (13937,) (3485,)


In [39]:
# X 값 Window Dataset 구성
ds_x = tf.data.Dataset.from_tensor_slices(SNE_X_train).window(WINDOW_SIZE, stride=1, shift=1, drop_remainder=True)  # 여기서는 X값만 별도로 구성하므로 WINDOW_SIZE 값에 1을 더하지 않는다!
ds_x = ds_x.flat_map(lambda x : x.batch(WINDOW_SIZE))

In [40]:
# y 값 Window Dataset 구성
ds_y = tf.data.Dataset.from_tensor_slices(SNE_y_train[WINDOW_SIZE:])

In [41]:
train_data = tf.data.Dataset.zip((ds_x, ds_y)).batch(BATCH_SIZE)

In [42]:
for x, y in train_data.take(1) :
  print(x[:3])
  print()
  print(y[:3])

tf.Tensor(
[[[-0.09386187  0.09445245 -0.66581843 -1.23511514]
  [-0.9225242   0.4051263  -0.66581843 -1.23511514]
  [-1.02610699  0.53827224 -0.66581843 -1.23511514]]

 [[-0.9225242   0.4051263  -0.66581843 -1.23511514]
  [-1.02610699  0.53827224 -0.66581843 -1.23511514]
  [-1.12968978  0.71580015 -0.66581843 -1.23511514]]

 [[-1.02610699  0.53827224 -0.66581843 -1.23511514]
  [-1.12968978  0.71580015 -0.66581843 -1.23511514]
  [-0.9225242   0.80456411 -0.66581843 -1.23511514]]], shape=(3, 3, 4), dtype=float64)

tf.Tensor([0. 0. 0.], shape=(3,), dtype=float64)


In [43]:
pd.concat([SNE_X_train, SNE_y_train], axis=1).head(10)

,1,3,4,5,8
0,-0.093862,0.094452,-0.665818,-1.235115,0.0
1,-0.922524,0.405126,-0.665818,-1.235115,0.0
2,-1.026107,0.538272,-0.665818,-1.235115,0.0
3,-1.129690,0.715800,-0.665818,-1.235115,0.0
4,-0.922524,0.804564,-0.665818,-1.235115,0.0
5,-1.026107,0.937710,-0.665818,-1.235115,0.0
6,-0.922524,0.893328,-0.665818,-1.235115,0.0
7,-0.922524,0.982092,0.549517,-1.235115,6000.0
8,-1.129690,0.538272,1.764852,0.769233,16000.0
9,-0.922524,0.005688,1.764852,-0.984572,24000.0


In [44]:
# 위 내용을 함수 형태로 만들고 싶은 경우

def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [45]:
train_data = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SNE_X_test, SNE_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [46]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 4), (28,)


In [47]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 4]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [48]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 4]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [49]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 4]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [50]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [51]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('new_multi_stne_lstm_weight_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('new_multi_stne_lstm_weight_ns_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('new_multi_stne_rnn_weight_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('new_multi_stne_rnn_weight_ns_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('new_multi_stne_gru_weight_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('new_multi_stne_gru_weight_ns_c4.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [52]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
    493/Unknown - 42s 8ms/step - loss: 7618.5762 - mse: 183546256.0000 - mae: 7618.9717
Epoch 1: val_loss improved from inf to 7580.27197, saving model to new_multi_stne_lstm_weight_c4.h5
498/498 [==============================] - 46s 16ms/step - loss: 7613.2754 - mse: 183255968.0000 - mae: 7613.6699 - val_loss: 7580.2720 - val_mse: 181875584.0000 - val_mae: 7580.6890 - lr: 0.0010
Epoch 2/500
498/498 [==============================] - ETA: 0s - loss: 7523.5278 - mse: 179329520.0000 - mae: 7523.9131
Epoch 2: val_loss improved from 7580.27197 to 7449.39111, saving model to new_multi_stne_lstm_weight_c4.h5
498/498 [==============================] - 6s 12ms/step - loss: 7523.5278 - mse: 179329520.0000 - mae: 7523.9131 - val_loss: 7449.3911 - val_mse: 176235632.0000 - val_mae: 7449.8218 - lr: 0.0010
Epoch 3/500
497/498 [============================>.] - ETA: 0s - loss: 7355.7578 - mse: 172288512.0000 - mae: 7356.1333
Epoch 3: val_loss improved from 7449.39111 to 7247.44580, savi

In [53]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
    492/Unknown - 5s 3ms/step - loss: 7599.9111 - mse: 183075024.0000 - mae: 7600.2817
Epoch 1: val_loss improved from inf to 7598.99414, saving model to new_multi_stne_rnn_weight_c4.h5
498/498 [==============================] - 7s 6ms/step - loss: 7617.1475 - mse: 183435664.0000 - mae: 7617.5186 - val_loss: 7598.9941 - val_mse: 182562688.0000 - val_mae: 7599.4019 - lr: 0.0010
Epoch 2/500
498/498 [==============================] - ETA: 0s - loss: 7565.6484 - mse: 181130656.0000 - mae: 7566.0205
Epoch 2: val_loss improved from 7598.99414 to 7527.98730, saving model to new_multi_stne_rnn_weight_c4.h5
498/498 [==============================] - 3s 5ms/step - loss: 7565.6484 - mse: 181130656.0000 - mae: 7566.0205 - val_loss: 7527.9873 - val_mse: 179376880.0000 - val_mae: 7528.3760 - lr: 0.0010
Epoch 3/500
497/498 [============================>.] - ETA: 0s - loss: 7482.1162 - mse: 177444160.0000 - mae: 7482.4854
Epoch 3: val_loss improved from 7527.98730 to 7420.26904, saving mod

In [54]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
    488/Unknown - 10s 5ms/step - loss: 7624.7451 - mse: 184011152.0000 - mae: 7625.1182
Epoch 1: val_loss improved from inf to 7614.74902, saving model to new_multi_stne_gru_weight_c4.h5
498/498 [==============================] - 12s 10ms/step - loss: 7624.5332 - mse: 183781440.0000 - mae: 7624.9072 - val_loss: 7614.7490 - val_mse: 183365200.0000 - val_mae: 7615.1523 - lr: 0.0010
Epoch 2/500
492/498 [============================>.] - ETA: 0s - loss: 7597.8159 - mse: 182759232.0000 - mae: 7598.1890
Epoch 2: val_loss improved from 7614.74902 to 7583.62012, saving model to new_multi_stne_gru_weight_c4.h5
498/498 [==============================] - 4s 7ms/step - loss: 7597.8179 - mse: 182644624.0000 - mae: 7598.1914 - val_loss: 7583.6201 - val_mse: 181721744.0000 - val_mae: 7584.0552 - lr: 0.0010
Epoch 3/500
491/498 [============================>.] - ETA: 0s - loss: 7529.6836 - mse: 180154512.0000 - mae: 7530.0586
Epoch 3: val_loss improved from 7583.62012 to 7527.31445, saving 

In [55]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 452.9240417480469
lstm_history Validation MSE: 3060805.5
lstm_history Validation MAE: 453.2364807128906
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 374.90283203125
lstm_history_ns Validation MSE: 2877303.0
lstm_history_ns Validation MAE: 375.2120666503906
-------------rnn_history-------------
rnn_history Validation Loss: 1075.875732421875
rnn_history Validation MSE: 8529425.0
rnn_history Validation MAE: 1076.213134765625
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 992.7092895507812
rnn_history_ns Validation MSE: 8058551.0
rnn_history_ns Validation MAE: 993.0484008789062
-------------gru_history-------------
gru_history Validation Loss: 1074.9676513671875
gru_history Validation MSE: 8506983.0
gru_history Validation MAE: 1075.31640625
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 1089.517333984375
gru_history_ns Validation MSE: 862139

In [56]:
lstm_save_path = f"./Models/lstm_model_stne_c4"
rnn_save_path = f"./Models/rnn_model_stne_c4"
gru_save_path = f"./Models/gru_model_stne_c4"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_c4\assets


INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_c4\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_c4\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_c4\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_c4\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_c4\assets


In [57]:
St_NotEncode_Basic_data = pd.read_pickle("Basic_StandardScalar_final_data")
  
print(St_NotEncode_Basic_data.shape)

(52578, 9)


In [58]:
# Feature와 Label 분리하기 (풍속, 습도, 일조시간, 일사량, 전운량)
def Feature_Label(datafile) :
    X = datafile.iloc[:,[1,3,4,5,6]]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SB_X, SB_y = Feature_Label(St_NotEncode_Basic_data)
print(SNE_X.shape, SNE_y.shape)
SB_X_train, SB_X_test, SB_y_train, SB_y_test = train_test_split(SB_X, SB_y, test_size=0.2, random_state=10, shuffle=False)
print(SB_X_train.shape, SB_X_test.shape, SB_y_train.shape, SB_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 4) (17422,)
(42062, 5) (10516, 5) (42062,) (10516,)


In [59]:
def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [60]:
train_data = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SB_X_test, SB_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [61]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 5), (28,)


In [62]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 5]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [63]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 5]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [64]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 5]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [65]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [66]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('base_multi_stne_lstm_weight_c5.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('base_multi_stne_lstm_weight_ns_c5.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('base_multi_stne_rnn_weight_c5.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('base_multi_stne_rnn_weight_ns_c5.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('base_multi_stne_gru_weight_c5.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('base_multi_stne_gru_weight_ns_c5.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [67]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
   1502/Unknown - 41s 10ms/step - loss: 1.9070 - mse: 20.0100 - mae: 2.2660
Epoch 1: val_loss improved from inf to 2.17016, saving model to base_multi_stne_lstm_weight_c5.h5
1503/1503 [==============================] - 47s 14ms/step - loss: 1.9069 - mse: 20.0086 - mae: 2.2659 - val_loss: 2.1702 - val_mse: 19.3776 - val_mae: 2.5825 - lr: 0.0010
Epoch 2/500
1502/1503 [============================>.] - ETA: 0s - loss: 1.3422 - mse: 9.6914 - mae: 1.6880
Epoch 2: val_loss improved from 2.17016 to 2.00707, saving model to base_multi_stne_lstm_weight_c5.h5
1503/1503 [==============================] - 17s 11ms/step - loss: 1.3423 - mse: 9.6925 - mae: 1.6881 - val_loss: 2.0071 - val_mse: 18.9786 - val_mae: 2.3929 - lr: 0.0010
Epoch 3/500
1503/1503 [==============================] - ETA: 0s - loss: 1.2879 - mse: 9.2160 - mae: 1.6336
Epoch 3: val_loss improved from 2.00707 to 2.00688, saving model to base_multi_stne_lstm_weight_c5.h5
1503/1503 [==============================] - 18s 12

In [68]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
   1497/Unknown - 10s 4ms/step - loss: 1.8379 - mse: 18.3110 - mae: 2.1994
Epoch 1: val_loss improved from inf to 2.20494, saving model to base_multi_stne_rnn_weight_c5.h5
1503/1503 [==============================] - 12s 5ms/step - loss: 1.8392 - mse: 18.3216 - mae: 2.2008 - val_loss: 2.2049 - val_mse: 22.0055 - val_mae: 2.5830 - lr: 0.0010
Epoch 2/500
1501/1503 [============================>.] - ETA: 0s - loss: 1.3226 - mse: 9.5323 - mae: 1.6724
Epoch 2: val_loss improved from 2.20494 to 2.14607, saving model to base_multi_stne_rnn_weight_c5.h5
1503/1503 [==============================] - 7s 4ms/step - loss: 1.3239 - mse: 9.5757 - mae: 1.6737 - val_loss: 2.1461 - val_mse: 20.4481 - val_mae: 2.5695 - lr: 0.0010
Epoch 3/500
1502/1503 [============================>.] - ETA: 0s - loss: 1.3027 - mse: 9.3225 - mae: 1.6490
Epoch 3: val_loss improved from 2.14607 to 2.07900, saving model to base_multi_stne_rnn_weight_c5.h5
1503/1503 [==============================] - 7s 4ms/step -

In [69]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
   1500/Unknown - 24s 9ms/step - loss: 2.0156 - mse: 22.4212 - mae: 2.3735
Epoch 1: val_loss improved from inf to 2.25509, saving model to base_multi_stne_gru_weight_c5.h5
1503/1503 [==============================] - 28s 12ms/step - loss: 2.0148 - mse: 22.4040 - mae: 2.3727 - val_loss: 2.2551 - val_mse: 23.3162 - val_mae: 2.6350 - lr: 0.0010
Epoch 2/500
1503/1503 [==============================] - ETA: 0s - loss: 1.2978 - mse: 9.3650 - mae: 1.6387
Epoch 2: val_loss improved from 2.25509 to 1.94440, saving model to base_multi_stne_gru_weight_c5.h5
1503/1503 [==============================] - 15s 10ms/step - loss: 1.2978 - mse: 9.3650 - mae: 1.6387 - val_loss: 1.9444 - val_mse: 18.4045 - val_mae: 2.3109 - lr: 0.0010
Epoch 3/500
1497/1503 [============================>.] - ETA: 0s - loss: 1.2611 - mse: 9.0295 - mae: 1.5967
Epoch 3: val_loss did not improve from 1.94440
1503/1503 [==============================] - 15s 10ms/step - loss: 1.2629 - mse: 9.0497 - mae: 1.5986 - val_l

In [70]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1.7406405210494995
lstm_history Validation MSE: 16.33970832824707
lstm_history Validation MAE: 2.0940611362457275
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 1.753144383430481
lstm_history_ns Validation MSE: 16.40740966796875
lstm_history_ns Validation MAE: 2.1096251010894775
-------------rnn_history-------------
rnn_history Validation Loss: 1.7402321100234985
rnn_history Validation MSE: 16.59107208251953
rnn_history Validation MAE: 2.1083638668060303
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 1.841468095779419
rnn_history_ns Validation MSE: 17.441804885864258
rnn_history_ns Validation MAE: 2.203950881958008
-------------gru_history-------------
gru_history Validation Loss: 1.7526441812515259
gru_history Validation MSE: 16.53897476196289
gru_history Validation MAE: 2.112900495529175
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 1.8

In [71]:
lstm_save_path = f"./Models/lstm_model_stne_base_c5"
rnn_save_path = f"./Models/rnn_model_stne_base_c5"
gru_save_path = f"./Models/gru_model_stne_base_c5"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_base_c5\assets


INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_base_c5\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_base_c5\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_base_c5\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_base_c5\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_base_c5\assets
